# Core Loss Analysis with PyOpenMagnetics

This notebook explores core loss calculations in depth, including:
- Different loss models (Steinmetz, iGSE, MSE)
- Temperature effects
- Waveform analysis
- Material comparison

In [ ]:
import PyOpenMagnetics
import json
from pprint import pprint

try:
    import matplotlib.pyplot as plt
    import numpy as np
    HAS_MATPLOTLIB = True
except ImportError:
    HAS_MATPLOTLIB = False
    print("Install matplotlib for visualizations: pip install matplotlib numpy")

## 1. Core Loss Fundamentals

Core losses consist of:
- **Hysteresis losses** - energy lost reorienting magnetic domains
- **Eddy current losses** - induced currents in the core material
- **Residual losses** - other frequency-dependent losses

The Steinmetz equation approximates these as:

$$ P_v = k \cdot f^\alpha \cdot B^\beta $$

Where:
- $P_v$ is volumetric power loss (W/m³)
- $f$ is frequency (Hz)
- $B$ is peak flux density (T)
- $k$, $\alpha$, $\beta$ are material-specific constants

In [3]:
# Create a test core
core_data = {
    "functionalDescription": {
        "name": "Test Core",
        "type": "two-piece set",
        "shape": "E 42/21/15",
        "material": "3C95",
        "gapping": [],
        "numberStacks": 1
    }
}

# API: calculate_core_data(core_data, include_material_data)
core = PyOpenMagnetics.calculate_core_data(core_data, False)
volume = core['processedDescription']['effectiveParameters']['effectiveVolume']
print(f"Core shape: E 42/21/15")
print(f"Material: 3C95")
print(f"Effective volume: {volume * 1e6:.2f} cm³")

NameError: name 'PyOpenMagnetics' is not defined

## 2. Steinmetz Coefficient Analysis

The Steinmetz equation coefficients vary with frequency. Let's analyze how they change.

In [2]:
# Get Steinmetz coefficients at different frequencies
frequencies = [25000, 50000, 100000, 200000, 500000]
material = "3C95"

print(f"Steinmetz coefficients for {material}:\n")
print(f"{'Frequency':>12} {'k':>12} {'alpha':>8} {'beta':>8}")
print("-" * 44)

steinmetz_data = []
for freq in frequencies:
    try:
        coeffs = PyOpenMagnetics.get_core_material_steinmetz_coefficients(material, float(freq))
        k = coeffs.get('k', 'N/A')
        alpha = coeffs.get('alpha', 'N/A')
        beta = coeffs.get('beta', 'N/A')
        
        if isinstance(k, (int, float)) and isinstance(alpha, (int, float)):
            print(f"{freq/1000:>10.0f} kHz {k:>12.6f} {alpha:>8.3f} {beta:>8.3f}")
            steinmetz_data.append((freq, k, alpha, beta))
        else:
            print(f"{freq/1000:>10.0f} kHz  {'Data not available':>30}")
    except Exception as e:
        print(f"{freq/1000:>10.0f} kHz  Error: {e}")

Steinmetz coefficients for 3C95:

   Frequency            k    alpha     beta
--------------------------------------------
        25 kHz  Error: name 'PyOpenMagnetics' is not defined
        50 kHz  Error: name 'PyOpenMagnetics' is not defined
       100 kHz  Error: name 'PyOpenMagnetics' is not defined
       200 kHz  Error: name 'PyOpenMagnetics' is not defined
       500 kHz  Error: name 'PyOpenMagnetics' is not defined


In [ ]:
# Calculate estimated volumetric losses using Steinmetz equation
# P_v = k * f^alpha * B^beta
B_peak = 0.2  # 200 mT peak flux density
volume_m3 = volume  # From core calculation above

if steinmetz_data:
    print(f"\nEstimated core losses at B_peak = {B_peak*1000:.0f} mT:\n")
    print(f"{'Frequency':>12} {'P_v (kW/m³)':>15} {'P_core (W)':>12}")
    print("-" * 42)
    
    for freq, k, alpha, beta in steinmetz_data:
        # Steinmetz equation: P_v = k * f^alpha * B^beta
        P_v = k * (freq ** alpha) * (B_peak ** beta)
        P_core = P_v * volume_m3
        print(f"{freq/1000:>10.0f} kHz {P_v/1000:>15.2f} {P_core:>12.3f}")
else:
    print("No Steinmetz data available for plotting")

## 3. Material Comparison

Compare loss characteristics of different ferrite materials.

In [ ]:
# Compare materials at fixed operating point
frequency = 100000  # 100 kHz
B_peak = 0.2        # 200 mT

materials_to_compare = ["3C90", "3C95", "3C97", "N87", "N97"]
print(f"Material comparison at f = {frequency/1000:.0f} kHz, B = {B_peak*1000:.0f} mT:\n")

comparison_data = []
for mat in materials_to_compare:
    try:
        coeffs = PyOpenMagnetics.get_core_material_steinmetz_coefficients(mat, float(frequency))
        k = coeffs.get('k')
        alpha = coeffs.get('alpha')
        beta = coeffs.get('beta')
        
        if k and alpha and beta:
            P_v = k * (frequency ** alpha) * (B_peak ** beta)
            comparison_data.append((mat, P_v, k, alpha, beta))
            print(f"{mat}: P_v = {P_v/1000:.2f} kW/m³")
        else:
            print(f"{mat}: Coefficients not available")
    except Exception as e:
        print(f"{mat}: Error - {e}")

In [ ]:
# Visualize material comparison
if HAS_MATPLOTLIB and comparison_data:
    materials = [d[0] for d in comparison_data]
    p_values = [d[1]/1000 for d in comparison_data]  # kW/m³
    
    plt.figure(figsize=(10, 6))
    bars = plt.bar(materials, p_values, color=['#3498db', '#2ecc71', '#9b59b6', '#e74c3c', '#f1c40f'][:len(materials)])
    plt.xlabel('Material', fontsize=12)
    plt.ylabel('Volumetric Power Loss (kW/m³)', fontsize=12)
    plt.title(f'Core Loss Comparison at {frequency/1000:.0f} kHz, {B_peak*1000:.0f} mT', fontsize=14)
    plt.grid(True, axis='y', alpha=0.3)
    
    # Add value labels on bars
    for bar, val in zip(bars, p_values):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                f'{val:.1f}', ha='center', va='bottom', fontsize=10)
    
    plt.tight_layout()
    plt.show()
else:
    print("No data available for visualization")

## 4. Permeability vs Frequency

Material permeability decreases at high frequencies.

In [ ]:
# Get permeability vs frequency for different materials
frequencies = [10000, 25000, 50000, 100000, 200000, 500000, 1000000]
temperature = 25.0
dc_bias = 0.0

permeability_data = {}
for mat in ["3C95", "N87"]:
    permeability_data[mat] = []
    for freq in frequencies:
        try:
            perm = PyOpenMagnetics.get_material_permeability(mat, temperature, dc_bias, float(freq))
            permeability_data[mat].append(perm)
        except:
            permeability_data[mat].append(None)

# Print table
print(f"Relative Permeability vs Frequency (at {temperature}°C, no DC bias):\n")
print(f"{'Frequency':>12} {'3C95':>10} {'N87':>10}")
print("-" * 34)
for i, freq in enumerate(frequencies):
    p1 = permeability_data.get("3C95", [None])[i]
    p2 = permeability_data.get("N87", [None])[i]
    p1_str = f"{p1:.0f}" if p1 else "N/A"
    p2_str = f"{p2:.0f}" if p2 else "N/A"
    print(f"{freq/1000:>10.0f} kHz {p1_str:>10} {p2_str:>10}")

In [ ]:
# Visualize permeability vs frequency
if HAS_MATPLOTLIB:
    plt.figure(figsize=(10, 6))
    
    for mat, perms in permeability_data.items():
        valid_freqs = [f/1000 for f, p in zip(frequencies, perms) if p is not None]
        valid_perms = [p for p in perms if p is not None]
        
        if valid_perms:
            plt.semilogx(valid_freqs, valid_perms, '-o', linewidth=2, markersize=8, label=mat)
    
    plt.xlabel('Frequency (kHz)', fontsize=12)
    plt.ylabel('Relative Permeability', fontsize=12)
    plt.title('Permeability vs Frequency', fontsize=14)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## 5. Loss Model Information

Get information about available loss models for materials.

In [ ]:
# Get available loss calculation methods for a material
material = PyOpenMagnetics.find_core_material_by_name("3C95")
methods = PyOpenMagnetics.get_core_material_available_losses_methods(material)

print(f"Available loss calculation methods for 3C95:")
for method in methods:
    print(f"  - {method}")

# Get detailed material info
print(f"\nMaterial details:")
print(f"  Name: {material.get('name', 'N/A')}")
print(f"  Manufacturer: {material.get('manufacturer', 'N/A')}")
if 'curiTemperature' in material:
    print(f"  Curie temperature: {material['curiTemperature']}°C")

In [ ]:
# Summary of analysis
print("=" * 50)
print("CORE LOSS ANALYSIS SUMMARY")
print("=" * 50)
print(f"""
Key Takeaways:

1. Steinmetz Equation: P = k × f^α × B^β
   - k, α, β vary with frequency range
   - Typical α ≈ 1.2-1.5, β ≈ 2.0-2.5 for ferrites

2. Material Selection:
   - Lower loss at target frequency = better efficiency
   - Consider permeability roll-off at high frequencies
   - Check saturation flux density limits

3. Operating Frequency:
   - Losses increase roughly with f^1.3 to f^1.5
   - Higher frequency = smaller core, but more losses

4. Temperature Effects:
   - Most ferrites have a loss minimum around 80-100°C
   - Permeability drops at high temperatures

For complete loss calculations including winding effects,
use the calculate_core_losses() and calculate_winding_losses() APIs
with full magnetic component definitions.
""")

In [ ]:
if HAS_MATPLOTLIB:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    names = list(waveform_results.keys())
    values = list(waveform_results.values())
    colors = ['#2ecc71', '#3498db', '#e74c3c', '#9b59b6']
    
    bars = ax.bar(names, values, color=colors, edgecolor='black', linewidth=1.2)
    
    # Add value labels
    for bar, val in zip(bars, values):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                f'{val:.3f} W', ha='center', va='bottom', fontsize=11)
    
    ax.set_ylabel('Core Losses (W)', fontsize=12)
    ax.set_title('Core Losses for Different Waveforms\n(E 42/21/15, 3C95, 100 kHz, 200 mT peak)', fontsize=14)
    ax.set_xticklabels(names, rotation=15, ha='right')
    plt.tight_layout()
    plt.show()

## Summary

Key takeaways:

1. **Core losses increase non-linearly** with both flux density and frequency
2. **Temperature affects losses** - many materials have a loss minimum around 80-100°C
3. **Material selection is critical** - different materials are optimized for different frequencies
4. **Waveform matters** - non-sinusoidal waveforms generally have higher losses

## Next Steps

- Try the **04_core_adviser.ipynb** notebook to find optimal cores automatically
- Explore **05_winding_losses.ipynb** for complete loss analysis